# napari

In [ ]:
import numpy as np
import napari
import copy
import pickle
import nest_asyncio
nest_asyncio.apply()
import torch
import torchvision
import timm
from pathml.core import HESlide
from sklearn.decomposition import PCA
from pathml.preprocessing import Pipeline, BoxBlur, TissueDetectionHE
import csv
from sklearn.cluster import KMeans
from magicgui import magicgui

In C:\Users\lhy\AppData\Local\Continuum\anaconda3\envs\pathml1\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\lhy\AppData\Local\Continuum\anaconda3\envs\pathml1\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\lhy\AppData\Local\Continuum\anaconda3\envs\pathml1\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\lhy\AppData\Local\Continuum\anaconda3\envs\pathml1\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.

In [2]:
def generate_pca_point(row,matrix,point_size):
    #print(matrix[0:10])
    matrix_min=np.min(matrix,axis=0)
    matrix=matrix-np.min(matrix,axis=0)
    matrix_max=np.max(matrix,axis=0)
    col=int(row*matrix_max[1]/matrix_max[0])
    plot_matrix=np.ones((row,col,3),dtype='uint8')*255
    min_col=0.05*col
    min_row=0.05*row
    max_row=0.95*row
    max_col=0.95*col
    matrix[:,0]=min_row+matrix[:,0]*(max_row-min_row)/matrix_max[0]
    matrix[:,1]=min_col+matrix[:,1]*(max_col-min_col)/matrix_max[1]
    matrix=matrix.astype(int)
    width=max(int(min(row,col)*0.002),1)
    row_interval=int(row/4)
    col_interval=int(col/4)
    color=220
    for i in range(1,4):
        plot_matrix[i*row_interval:i*row_interval+width,:]=color
    for i in range(1,4):
        plot_matrix[:,i*col_interval:i*col_interval+width,:]=color
    return plot_matrix,matrix,min_col,min_row,max_row,max_col,matrix_min,matrix_max
def transform_origpos(x,y,row,col,orig_row,orig_col,plot_col):
    x1=x
    y1=y
    x1=int(x1*row/orig_row)
    y1=int(y1*col/orig_col)
    y1=y1+plot_col
    return x1,y1
def readincoord(file_path):
    coord=[]
    with open(file_path) as file:
        reader=csv.reader(file)
        for i in reader:
            coord.append(i)
    coord=np.array(coord,dtype=int)
    coord=[list(coord[i]) for i in range(coord.shape[0])]
    return coord
def gen_cluster_mask(coords,row,col,orig_row,orig_col,plot_col,tile_size):
    polys=[]
    for i in range(len(coords)):
        media_x,media_y=transform_origpos(coords[i][0],coords[i][1],row,col,orig_row,orig_col,plot_col)
        media1_x,media1_y=transform_origpos(coords[i][0]+tile_size,coords[i][1]+tile_size,row,col,orig_row,orig_col,plot_col)
        media=np.array([[media_x,media_y],[media_x,media1_y],[media1_x,media1_y],[media1_x,media_y]],dtype=int)
        polys.append(media)
    return polys
def gen_mask(coords,row,col,orig_row,orig_col,plot_col,tile_size,line_width):
    lines=[]
    ld=0.5*float(line_width)
    for i in range(len(coords)):
        media_x,media_y=transform_origpos(coords[i][0],coords[i][1],row,col,orig_row,orig_col,plot_col)
        media1_x,media1_y=transform_origpos(coords[i][0]+tile_size,coords[i][1]+tile_size,row,col,orig_row,orig_col,plot_col)
        media=np.array([[media_x,media_y-ld],[media_x,media1_y],[media1_x,media1_y],[media1_x,media_y],[media_x-ld,media_y]])
        lines.append(media)
    return lines

In [3]:
MODEL_PATH = 'C:/harvard/capstone/tenpercent_resnet18.ckpt'
def load_model_weights(model, weights):

    model_dict = model.state_dict()
    weights = {k: v for k, v in weights.items() if k in model_dict}
    if weights == {}:
        print('No weight could be loaded..')
    model_dict.update(weights)
    model.load_state_dict(model_dict)

    return model

model1 = torchvision.models.__dict__['resnet18'](pretrained=False)

state = torch.load(MODEL_PATH, map_location=torch.device('cpu'))

state_dict = state['state_dict']
for key in list(state_dict.keys()):
    state_dict[key.replace('model.', '').replace('resnet.', '')] = state_dict.pop(key)

model1 = load_model_weights(model1, state_dict)
model1.fc = torch.nn.Sequential()

In [4]:
n_clusters=5
tile_ma=[]
with open("C:/harvard/capstone/tilefeatures2_eval6.csv") as file:
    reader=csv.reader(file)
    for i in reader:
        tile_ma.append(i)
tissue_masks=[]
with open("C:/harvard/capstone/tissue_masks.csv") as file:
    reader=csv.reader(file)
    for i in reader:
        tissue_masks.append(int(i[0]))
coords=readincoord("C:/harvard/capstone/coordinates.csv")
tile_ma=tile_ma[1:]
assert len(coords)==len(tissue_masks)
assert len(tile_ma)==len(tissue_masks)
tile_ma=[tile_ma[i] for i in range(len(tile_ma)) if tissue_masks[i]>0]
coords=[coords[i] for i in range((len(coords))) if tissue_masks[i]>0]
tile_ma=np.array(tile_ma,dtype=float)
pca = PCA(n_components=2)
pca_matrix=pca.fit_transform(tile_ma)
kmean=KMeans(n_clusters=n_clusters, random_state=0).fit(tile_ma)

In [5]:
wsi = HESlide("C:/harvard/capstone/small_HE.svs", name = "example")
thumbnail = wsi.slide.get_thumbnail(size = (5000, 5000))
color_list=['green','blue','orange','black','purple','yellow','pink']
point_size=int(0.005*thumbnail.shape[0])
layer2_data,point_matrix,min_col,min_row,max_row,max_col,matrix_min,matrix_max=generate_pca_point(thumbnail.shape[0],pca_matrix,point_size)
tile_size=64
line_width=8
coords=readincoord("C:/harvard/capstone/filter_coords.csv")
polys=gen_cluster_mask(coords,thumbnail.shape[0],thumbnail.data.shape[1],wsi.shape[0],wsi.shape[1],layer2_data.shape[1],tile_size)
viewer = napari.Viewer()
p_color_list=[color_list[i] for i in kmean.labels_]
layer1=viewer.add_image(np.concatenate((layer2_data,thumbnail),axis=1),name='wsi')
layer2=viewer.add_points(point_matrix,size=point_size,properties={'coord_x':np.array(coords)[:,0],'coord_y':np.array(coords)[:,1]},edge_color=p_color_list,name='pca plot')
layer3=viewer.add_points(point_matrix[0:1],size=point_size*3,symbol='cross',opacity=0,edge_color='red',face_color='red',name='cross')
layer4=viewer.add_shapes(polys,properties={'coord_x':pca_matrix[:,0],'coord_y':pca_matrix[:,1]},edge_width=0,face_color=p_color_list,opacity=0.5,name='tile')
layer5=viewer.add_shapes(np.array([[1114,2997.5],[1114,3064],[1178,3064],[1178,3000],[1111.5,3000]]),edge_color='red',edge_width=line_width,shape_type='path',opacity=0.0,name='grid')
layer1.editable=False
layer3.editable=False
layer5.editable=False
def update_grid_position(event):
    if len(event.source.selected_data)>0:
        media=list(event.source.selected_data)
        coords=[layer4.data[i] for i in media]
        for i in range(len(coords)):
            coords[i]=np.concatenate((coords[i],coords[i][0:1]),axis=0)
            coords[i][0,1]=coords[i][0,1]-0.5*float(line_width)
            coords[i][4,0]=coords[i][4,0]-0.5*float(line_width)
        layer5.data=coords
        layer5.edge_width=[line_width]*len(coords)
        layer5.opacity=1.0
    else:
        layer5.opacity=0.0
layer2.events.mode.connect(update_grid_position)
def update_cross_position(event):
    if len(event.source.selected_data)>0:
        media=list(event.source.selected_data)
        coords=[layer2.data[i] for i in media]
        layer3.data=np.array(coords)
        layer3.edge_color=['red']*len(media)
        layer3.face_color=['red']*len(media)
        layer3.selected_data.clear()
        layer3.opacity=1.0
    else:
        layer3.selected_data.clear()
        layer3.opacity=0.0
layer4.events.mode.connect(update_cross_position)
@magicgui(slider_int={"widget_type": "Slider", "min": 1,'max':7,'label': 'Clusters of Kmeans'})
def slider(slider_int=5):
    kmean=KMeans(n_clusters=slider_int, random_state=0).fit(tile_ma)
    p_color_list=[color_list[i] for i in kmean.labels_]
    layer2.edge_color=p_color_list
    layer4.face_color=p_color_list
viewer.window.add_dock_widget(slider)
napari.run()

In [ ]:
wsi = HESlide("C:/harvard/capstone/small_HE.svs", name = "example")